In [ ]:
!pip install textacy

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM nlp_articles \
                   WHERE datePublished<'2021-01-01' \
                   ORDER BY datePublished", sql, index_col="id", parse_dates=["datePublished"])

In [ ]:
df.plot.scatter(x='no_tokens', y='no_sentences')

In [ ]:
import seaborn as sns
sns.jointplot(x=df["no_tokens"], y=df["no_sentences"], kind="reg")

In [ ]:
import scipy.stats
scipy.stats.pearsonr(df["no_tokens"], df["no_sentences"])

In [ ]:
count_length = df.groupby("author").agg({"url": "count", "no_tokens": "mean"}).rename(columns={"url": "count"})

In [ ]:
sns.jointplot(x=count_length["no_tokens"], y=count_length["count"], kind="reg")

In [ ]:
scipy.stats.pearsonr(count_length["no_tokens"], count_length["count"])

In [ ]:
sns.jointplot(x=df["no_tokens"].fillna(0).map(int), 
              y=df["commentCount"].fillna(0).map(int), kind="reg")

In [ ]:
scipy.stats.pearsonr(df["no_tokens"].fillna(0).map(int), df["commentCount"].fillna(0).map(int))

In [ ]:
top_authors = df.groupby("author").agg({"url": "count"}).sort_values("url").tail(20).index.values

In [ ]:
top_author_articles = df[df["author"].isin(top_authors)]
len(top_author_articles)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 10))
sns.boxplot(y="author", x="no_tokens", data=top_author_articles, palette="viridis")

In [ ]:
plt.figure(figsize=(8, 10))
labels = top_author_articles.groupby("author").agg({"no_tokens": "median"}).sort_values("no_tokens").index.values
sns.boxplot(y="author", x="no_tokens", data=top_author_articles, order=labels, palette="viridis")

In [ ]:
ben_schwan = df[df["author"] == "Ben Schwan"]

In [ ]:
mark_mantel = df[df["author"] == "Mark Mantel"]

In [ ]:
from collections import Counter
from wordcloud import WordCloud
from spacy.lang.de.stop_words import STOP_WORDS
import regex as re
def word_cloud_for_field(df, field):
    c = Counter([w for words in df[field] for w in re.split(r'\||\#', words)])
    for w in STOP_WORDS:
        c[w] = 0
    wc = WordCloud(background_color="white", max_words=100)
    wc.generate_from_frequencies(c)
    plt.figure(figsize=(12,12))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis("off");    

In [ ]:
word_cloud_for_field(ben_schwan, "nouns")

In [ ]:
word_cloud_for_field(mark_mantel, "nouns")